In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
cd C:\Users\anniw\IFT6758-A2022-G08

C:\Users\anniw\IFT6758-A2022-G08


In [7]:
import src.data.NHLDataManager as DataManager

In [8]:
nhl_data = DataManager.NHLDataManager()

Enter NHL_DATA_DIR environment:  C:\Users\anniw\OneDrive - Universite de Montreal\MSc 2022-2024\IFT6758 Science des données\nhl_data_dir


In [110]:
import os
import json
import pprint

import ipywidgets as widgets
from ipywidgets import interact

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.image as mpimg


@ interact(season_year=widgets.IntSlider(min=2016, max=2020, step=1, value=1, description='Year'), 
           season=widgets.Dropdown(options=['regular', 'playoffs'], value='regular',description='Season'))
def interactive_tool(season_year: int, season: str):
      
    dir_path = os.path.join(os.environ['NHL_DATA_DIR'], str(season_year), season)
    
    if season == 'regular':
        game_numbers = list(range(1, len(os.listdir(dir_path))+1))
    else: # read last three characters of file names instead of using the nested for loop cuz 2016 only has 102 files so the loop doesn't work 100%
        game_numbers = [int(os.path.splitext(filename)[0][-3:]) for filename in os.listdir(dir_path)]

    @ interact(game_id=widgets.SelectionSlider(options=game_numbers, value=115, description='Game ID'))
    def load_json(game_id: int): 
        
        # -- Build filename
        temp_game_id = str(game_id).zfill(4) 
        
        if season == 'regular':
            code = '02'
        else:
            code = '03' 
            
        file_path = os.path.join(dir_path, f'{season_year}{code}{temp_game_id}.json')
        
        # -- Open JSON file
        with open(file_path, 'r') as json_file:
              data = json.load(json_file)
        
        # -- Header
        print(data['gameData']['datetime']['dateTime'])
        
        abb_home = data['gameData']['teams']['home']['abbreviation']
        abb_away = data['gameData']['teams']['away']['abbreviation']
        print(f'Game ID: {game_id}; {abb_home} (home) vs {abb_away} (away)\n')
        
        # -- Table
        print('{}{}{}'.format(''.ljust(15), 'Home'.ljust(7), 'Away'.ljust(7)))
        print('{}{}{}'.format('Teams:'.ljust(15), abb_home.ljust(7), abb_away.ljust(7)))
        
        goals_home = str(data['liveData']['linescore']['teams']['home']['goals'])
        goals_away = str(data['liveData']['linescore']['teams']['away']['goals'])
        print('{}{}{}'.format('Goals:'.ljust(15), goals_home.ljust(7), goals_away.ljust(7)))

        sog_home = str(data['liveData']['linescore']['teams']['home']['shotsOnGoal'])
        sog_away = str(data['liveData']['linescore']['teams']['away']['shotsOnGoal'])
        print('{}{}{}'.format('SoG:'.ljust(15), sog_home.ljust(7), sog_away.ljust(7)))
         
        so_goals_home = str(data['liveData']['linescore']['shootoutInfo']['home']['scores'])
        so_goals_away = str(data['liveData']['linescore']['shootoutInfo']['away']['scores'])
        print('{}{}{}'.format('SO Goals:'.ljust(15), so_goals_home.ljust(7), so_goals_away.ljust(7)))
            
        so_attempts_home = str(data['liveData']['linescore']['shootoutInfo']['home']['attempts'])
        so_attempts_away = str(data['liveData']['linescore']['shootoutInfo']['away']['attempts'])
        print('{}{}{}'.format('SO Attempts:'.ljust(15), so_attempts_home.ljust(7), so_attempts_away.ljust(7)))
        
        @ interact(event_idx=widgets.IntSlider(min=0, max=max(0, len(data['liveData']['plays']['allPlays'])-1), step=1, value=0, description='Event Index'))
        def print_info(event_idx: int):
    
            plt.figure(figsize = (10, 5), dpi=100)
            img = mpimg.imread("https://raw.githubusercontent.com/udem-ift6758/project-template/main/figures/nhl_rink.png")
            plt.imshow(img, extent=[-100.0, 100.0, -42.5, 42.5])
            
            plt.xlabel('feet')
            plt.ylabel('feet')
            
            if data['gameData']['status']['abstractGameState'] != 'Preview':
                title1 = data['liveData']['plays']['allPlays'][event_idx]['result']['description'] 
                title2_0 = data['liveData']['plays']['allPlays'][event_idx]['about']['periodTime']
                title2_1 = data['liveData']['plays']['allPlays'][event_idx]['about']['period']
                plt.suptitle(f'{title1}\n{title2_0} P-{title2_1}')
            else:
                pass
            
            title3_0 = data['gameData']['teams']['away']['abbreviation']
            title3_1 = data['gameData']['teams']['home']['abbreviation']
            plt.title('{}{}'.format(title3_0.center(60), title3_1.center(60)), horizontalalignment='center')
              
            try: # plot event if exists
                x_coord = data['liveData']['plays']['allPlays'][event_idx]['coordinates']['x']
                y_coord = data['liveData']['plays']['allPlays'][event_idx]['coordinates']['y']
                plt.plot(x_coord, y_coord, marker='o', color='blue', markersize=10)
                
                plt.show()
                
                pprint.pprint(data['liveData']['plays']['allPlays'][event_idx])
            except:
                pass

interactive(children=(IntSlider(value=2016, description='Year', max=2020, min=2016), Dropdown(description='Sea…